[adaptado de [Programa de cursos integrados Aprendizado de máquina](https://www.coursera.org/specializations/machine-learning-introduction) de [Andrew Ng](https://www.coursera.org/instructor/andrewng)  ([Stanford University](http://online.stanford.edu/), [DeepLearning.AI](https://www.deeplearning.ai/) ) ]

In [ ]:
# Baixar arquivos adicionais para o laboratório
!wget https://github.com/fabiobento/dnn-course-2024-1/raw/main/00_course_folder/nn_adv/class_02/Laborat%C3%B3rios/lab_utils_ml_adv_add_week_2
      
!unzip -n -q lab_utils_ml_adv_add_week_2

In [ ]:
# Testar se estamos no Google Colab
# Necessário para ativar widgets
try:
  import google.colab
  IN_COLAB = True
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  IN_COLAB = False

# Propagação Posterior com Grafo Computacional (_Back Propagation_)
Ao trabalhar neste laboratório, você terá uma visão geral de um algoritmo importante usado pela maioria dos _frameworks_ de aprendizado de máquina.

A descida de gradiente requer a derivada do custo em relação a cada parâmetro da rede. As redes neurais podem ter milhões ou até bilhões de parâmetros. O algoritmo de *back propagation* é usado para calcular essas derivadas. *Os grafos computacionais são usados para simplificar a operação. Vamos nos aprofundar nisso a seguir.

In [ ]:
from sympy import *
import numpy as np
import re
%matplotlib widget
import matplotlib.pyplot as plt
from matplotlib.widgets import TextBox
from matplotlib.widgets import Button
import ipywidgets as widgets
from lab_utils_backprop import *

In [ ]:
!pip install ipympl

## Grafo Computacional
Um grafo computacional simplifica o cálculo de derivadas complexas, dividindo-as em etapas menores. Vamos ver como isso funciona.

Vamos calcular a derivada dessa expressão ligeiramente complexa, $J = (2+3w)^2$. Gostaríamos de encontrar a derivada de $J$ com relação a $w$ ou $\frac{\partial J}{\partial w}$.

In [ ]:
plt.close("all")
plt_network(config_nw0, "./images/C2_W2_BP_network0.PNG")

Acima, você pode ver que dividimos a expressão em dois nós, nos quais podemos trabalhar de forma independente. Se você já tem um bom entendimento do processo da aula, pode ir em frente e preencher as caixas no diagrama acima.

Primeiro, preencha as caixas azuis da esquerda para a direita e, em seguida, preencha as caixas verdes, começando à direita e indo para a esquerda.

Se você tiver os valores corretos, eles serão exibidos como verde ou azul. Se o valor estiver incorreto, ele será vermelho. Observe que o gráfico interativo não é particularmente robusto. Se você tiver problemas com a interface, execute a célula acima novamente para reiniciar.

Se você não tiver certeza do processo, trabalharemos nesse exemplo passo a passo a seguir.

### Propagação direta   
Vamos calcular os valores na propagação direta (_forward propagation_).

>Apenas uma observação sobre esta seção. Ela usa variáveis globais e as reutiliza à medida que o cálculo avança. Se você executar as células fora de ordem, poderá obter resultados estranhos. Se isso acontecer, volte a este ponto e execute-as em ordem.

In [ ]:
w = 3
a = 2+3*w
J = a**2
print(f"a = {a}, J = {J}")

Você pode preencher esses valores nas caixas azuis acima.

### _Backprop_
<img align="left" src="./images/C2_W2_BP_network0_j.PNG" style=" width:100px; padding: 10px 20px; " > _Backprop_ é o algoritmo que usamos para calcular as derivadas. Conforme descrito nas aulas, o _backprop_ começa à direita e se move para a esquerda. O primeiro nó a ser considerado é $J = a^2 $ e a primeira etapa é encontrar $\frac{\partial J}{\partial a}$ 


### $\frac{\partial J}{\partial a}$ 
#### Aritmeticamente
Encontre $\frac{\partial J}{\partial a}$ descobrindo como $J$ muda como resultado de uma pequena mudança em $a$. Isso é descrito em detalhes no laboratório de derivadas.

In [ ]:
a_epsilon = a + 0.001  # a mais um pequeno valor, epsilon
J_epsilon = a_epsilon**2
k = (J_epsilon - J)/0.001   # diferença dividida por epsilon
print(f"J = {J}, J_epsilon = {J_epsilon}, dJ_da ~= k = {k} ")

$\frac{\partial J}{\partial a}$ é 22, que é $2\times a$. Nosso resultado não é exatamente $2 \times a$ porque nosso valor de epsilon não é infinitesimalmente pequeno. 
#### Simbolicamente
Agora, vamos usar o SymPy para calcular as derivadas simbolicamente, como fizemos no laboratório sobre derivadas. Vamos prefixar o nome da variável com um "s" para indicar que se trata de uma variável *simbólica*.

In [ ]:
sw,sJ,sa = symbols('w,J,a')
sJ = sa**2
sJ

In [ ]:
sJ.subs([(sa,a)])

In [ ]:
dJ_da = diff(sJ, sa)
dJ_da

Portanto, $\frac{\partial J}{\partial a} = 2a$. Quando $a=11$, $\frac{\partial J}{\partial a} = 22$. Isso corresponde ao nosso cálculo aritmético acima.
Se ainda não tiver feito isso, você pode voltar ao diagrama acima e preencher o valor de $\frac{\partial J}{\partial a}$.

### $\frac{\partial J}{\partial w}$ 
<img align="left" src="./images/C2_W2_BP_network0_a.PNG"     style=" width:100px; padding: 10px 20px; " >

 Indo da direita para a esquerda, o próximo valor que gostaríamos de calcular é $\frac{\partial J}{\partial w}$. Para fazer isso, primeiro precisamos calcular $\frac{\partial a}{\partial w}$, que descreve como a saída desse nó, $a$, muda quando a entrada $w$ muda um pouco.

#### Aritmeticamente
Encontre $\frac{\partial a}{\partial w}$ descobrindo como $a$ muda como resultado de uma pequena mudança em $w$.

In [ ]:
w_epsilon = w + 0.001       # a mais um pequeno valor, epsilon
a_epsilon = 2 + 3*w_epsilon
k = (a_epsilon - a)/0.001   # diferença dividida por epsilon
print(f"a = {a}, a_epsilon = {a_epsilon}, da_dw ~= k = {k} ")

Calculado aritmeticamente, $\frac{\partial a}{\partial w} \approx 3$. Vamos tentar isso com o SymPy.

In [ ]:
sa = 2 + 3*sw
sa

In [ ]:
da_dw = diff(sa,sw)
da_dw

>A próxima etapa é a parte interessante:
> - Sabemos que uma pequena mudança em $w$ fará com que $a$ mude 3 vezes esse valor.
> - Sabemos que uma pequena mudança em $a$ fará com que $J$ mude em $2\times a$ vezes esse valor. (a=11 neste exemplo)    
 Então, juntando tudo isso, 
> Sabemos que uma pequena mudança em $w$ fará com que $J$ mude em $3 \times 2\times a$ vezes esse valor.
> 
> Essas alterações em cascata têm o nome de *regra da cadeia*.  Ela pode ser escrita da seguinte forma: 
 $$\frac{\partial J}{\partial w} = \frac{\partial a}{\partial w} \frac{\partial J}{\partial a} $$
 
Se não estiver claro, vale a pena pensar um pouco sobre isso. Essa é a principal conclusão.
 
 Vamos tentar calculá-la:
 

In [ ]:
dJ_dw = da_dw * dJ_da
dJ_dw

E $a$ é 11 neste exemplo, portanto $\frac{\partial J}{\partial w} = 66$. Podemos verificar isso aritmeticamente:

In [ ]:
w_epsilon = w + 0.001
a_epsilon = 2 + 3*w_epsilon
J_epsilon = a_epsilon**2
k = (J_epsilon - J)/0.001   # diferença dividida por epsilon
print(f"J = {J}, J_epsilon = {J_epsilon}, dJ_dw ~= k = {k} ")

OK! Agora você pode preencher os valores de $\frac{\partial a}{\partial w}$ e $\frac{\partial J}{\partial w}$ no diagrama, caso ainda não o tenha feito. 

**Outra visão**  
É possível visualizar essas mudanças em cascata da seguinte forma:  

<img align="center" src="./images/C2_W2_BP_network0_diff.PNG"  style=" width:500px; padding: 10px 20px; " >  

Uma pequena alteração em $w$ é multiplicada por $\frac{\partial a}{\partial w}$, resultando em uma alteração que é 3 vezes maior. Essa alteração maior é então multiplicada por $\frac{\partial J}{\partial a}$, resultando em uma alteração que agora é $3 \times 22 = 66$ vezes maior.

## Gráfico Computacional de uma Rede Neural Simples
Abaixo está um gráfico da rede neural usada na aula com valores diferentes. Tente preencher os valores nas caixas. Observe que o gráfico interativo não é particularmente robusto. Se você tiver problemas com a interface, execute a célula abaixo novamente para reiniciar.

In [ ]:
plt.close("all")
plt_network(config_nw1, "./images/C2_W2_BP_network1.PNG")

A seguir, analisaremos detalhadamente os cálculos necessários para preencher o grafo computacional acima. Começaremos com o caminho para direto.

### Propagação direta(_Forward propagation_)
Os cálculos na Propagação direta são os que você aprendeu recentemente para redes neurais. Você pode comparar os valores abaixo com aqueles que calculou para o diagrama acima.

In [ ]:
# calcular valores por etapa 
x = 2
w = -2
b = 8
y = 1
  
c = w * x
a = c + b
d = a - y
J = d**2/2
print(f"J={J}, d={d}, a={a}, c={c}")

### _Backward propagation_ (Backprop)
<img align="left" src="./images/C2_W2_BP_network1_jdsq.PNG"     style=" width:100px; padding: 10px 20px; " >

Conforme descrito nas aulas, o backprop começa à direita e se move para a esquerda. O primeiro nó a ser considerado é $J = \frac{1}{2}d^2 $ e a primeira etapa é encontrar $\frac{\partial J}{\partial d}$ 


### $\frac{\partial J}{\partial d}$ 

#### Aritmeticamente
Encontre $\frac{\partial J}{\partial d}$ descobrindo como $J$ muda como resultado de uma pequena mudança em $d$.

In [ ]:
d_epsilon = d + 0.001
J_epsilon = d_epsilon**2/2
k = (J_epsilon - J)/0.001   # diferença dividida por epsilon
print(f"J = {J}, J_epsilon = {J_epsilon}, dJ_dd ~= k = {k} ")

$\frac{\partial J}{\partial d}$ é 3, que é o valor de $d$. Nosso resultado não é exatamente $d$ porque nosso valor de epsilon não é infinitesimalmente pequeno. 
#### Simbolicamente
Agora, vamos usar o SymPy para calcular as derivadas simbolicamente, como fizemos no laboratório opcional de derivadas.
O prefixo do nome da variável com um "s" indicará que se trata de uma variável *simbólica*.

In [ ]:
sx,sw,sb,sy,sJ = symbols('x,w,b,y,J')
sa, sc, sd = symbols('a,c,d')
sJ = sd**2/2
sJ

In [ ]:
sJ.subs([(sd,d)])

In [ ]:
dJ_dd = diff(sJ, sd)
dJ_dd

Portanto, $\frac{\partial J}{\partial d}$ = d. Quando $d=3$, $\frac{\partial J}{\partial d}$ = 3. Isso corresponde ao nosso cálculo aritmético acima.
Se ainda não tiver feito isso, você pode voltar ao diagrama acima e preencher o valor de $\frac{\partial J}{\partial d}$.

### $\frac{\partial J}{\partial a}$ 
<img align="left" src="./images/C2_W2_BP_network1_d.PNG"     style=" width:100px; padding: 10px 20px; " >

Indo da direita para a esquerda, o próximo valor que gostaríamos de calcular é $\frac{\partial J}{\partial a}$.

Para fazer isso, primeiro precisamos calcular $\frac{\partial d}{\partial a}$, que descreve como a saída desse nó muda quando a entrada $a$ muda um pouco. (Observe que não estamos interessados em como a saída muda quando $y$ muda, pois $y$ não é um parâmetro).

#### Aritmeticamente
Encontre $\frac{\partial d}{\partial a}$ descobrindo como $d$ muda como resultado de uma pequena mudança em $a$.

In [ ]:
a_epsilon = a + 0.001         #a mais um pequeno valor
d_epsilon = a_epsilon - y
k = (d_epsilon - d)/0.001   # diferença dividida por epsilon
print(f"d = {d}, d_epsilon = {d_epsilon}, dd_da ~= k = {k} ")

Calculado aritmeticamente, $\frac{\partial d}{\partial a} \approx 1$. Vamos tentar isso com o SymPy.
#### Simbolicamente

In [ ]:
sd = sa - sy
sd

In [ ]:
dd_da = diff(sd,sa)
dd_da

Calculado aritmeticamente, $\frac{\partial d}{\partial a}$ também é igual a 1.  
>A próxima etapa é a parte interessante, repetida novamente neste exemplo:
> - Sabemos que uma pequena alteração em $a$ fará com que $d$ seja alterado em 1 vez esse valor.
> - Sabemos que uma pequena mudança em $d$ fará com que $J$ mude em $d$ vezes esse valor. (d=3 neste exemplo)    
 Então, juntando tudo isso, 
> Sabemos que uma pequena mudança em $a$ fará com que $J$ mude em $1\times d$ vezes esse valor.
> 
>Isso é novamente a *regra da cadeia*.  Ela pode ser escrita da seguinte forma: 
 $$\frac{\partial J}{\partial a} = \frac{\partial d}{\partial a} \frac{\partial J}{\partial d} $$
 
 Vamos tentar calcular isso:
 
 

In [ ]:
dJ_da = dd_da * dJ_dd
dJ_da

E $d$ é 3 neste exemplo, portanto $\frac{\partial J}{\partial a} = 3$. Podemos verificar isso aritmeticamente:

In [ ]:
a_epsilon = a + 0.001
d_epsilon = a_epsilon - y
J_epsilon = d_epsilon**2/2
k = (J_epsilon - J)/0.001   
print(f"J = {J}, J_epsilon = {J_epsilon}, dJ_da ~= k = {k} ")

OK, eles coincidem! Agora você pode preencher os valores de $\frac{\partial d}{\partial a}$ e $\frac{\partial J}{\partial a}$ no diagrama, caso ainda não o tenha feito. 

> **As etapas do backprop**   
>Agora que você já trabalhou com vários nós, podemos escrever o método básico:\
> Trabalhando da direita para a esquerda, para cada nó:
>- calcular a(s) derivada(s) local(is) do nó
>- usando a regra da cadeia, combine com a derivada do custo com relação ao nó à direita.   

A(s) "derivada(s) local(is)" é(são) a(s) derivada(s) da saída do nó atual com relação a todas as entradas ou parâmetros.

Vamos continuar o trabalho. Seremos um pouco menos prolixos agora que você está familiarizado com o método.

### $\frac{\partial J}{\partial c}$,  $\frac{\partial J}{\partial b}$
<img align="left" src="./images/C2_W2_BP_network1_a.PNG"     style=" width:100px; padding: 10px 20px; " >

O próximo nó tem duas derivadas de interesse. Precisamos calcular $\frac{\partial J}{\partial c}$ para que possamos nos propagar para a esquerda. Também queremos calcular $\frac{\partial J}{\partial b}$.

Encontrar a derivada do custo com relação aos parâmetros $w$ e $b$ é o objetivo do backprop. Encontraremos as derivadas locais, $\frac{\partial a}{\partial c}$ e $\frac{\partial a}{\partial b}$ primeiro e, em seguida, combinaremos essas derivadas com a derivada vinda da direita, $\frac{\partial J}{\partial a}$.

In [ ]:
# calcular as derivadas locais da_dc, da_db
sa = sc + sb
sa

In [ ]:
da_dc = diff(sa,sc)
da_db = diff(sa,sb)
print(da_dc, da_db)

In [ ]:
dJ_dc = da_dc * dJ_da
dJ_db = da_db * dJ_da
print(f"dJ_dc = {dJ_dc},  dJ_db = {dJ_db}")

E, em nosso exemplo, d = 3

###  $\frac{\partial J}{\partial w}$
<img align="left" src="./images/C2_W2_BP_network1_c.PNG"     style=" width:100px; padding: 10px 20px; " >

O último nó deste exemplo calcula `c`. Aqui, estamos interessados em como J muda em relação ao parâmetro w. Não faremos a retropropagação para a entrada $x$, portanto, não estamos interessados em $\frac{\partial J}{\partial x}$. Vamos começar calculando $\frac{\partial c}{\partial w}$.

In [ ]:
# calcular a derivada local
sc = sw * sx
sc

In [ ]:
dc_dw = diff(sc,sw)
dc_dw

Essa derivada é um pouco mais interessante do que a anterior. Ela varia de acordo com o valor de $x$. Em nosso exemplo, esse valor é 2.

Combine isso com $\frac{\partial J}{\partial c}$ para encontrar $\frac{\partial J}{\partial w}$.

In [ ]:
dJ_dw = dc_dw * dJ_dc
dJ_dw

In [ ]:
print(f"dJ_dw = {dJ_dw.subs([(sd,d),(sx,x)])}")

$d=3$, portanto $\frac{\partial J}{\partial w} = 6$ para o nosso exemplo.   
Vamos testar isso aritmeticamente:

In [ ]:
J_epsilon = ((w+0.001)*x+b - y)**2/2
k = (J_epsilon - J)/0.001  
print(f"J = {J}, J_epsilon = {J_epsilon}, dJ_dw ~= k = {k} ")

Eles combinam! Ótimo. Você pode adicionar $\frac{\partial J}{\partial w}$ ao diagrama acima e nossa análise estará completa.

## Parabéns!
Você trabalhou em um exemplo de retropropagação usando um grafo computacional. Você pode aplicar isso a exemplos maiores seguindo a mesma abordagem nó a nó. 